In [1]:
# imports
import pandas as pd
import json 
import requests
from datetime import datetime
import numpy as np 
import os 
from pathlib import Path  
import re

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [3]:
df_bike = pd.read_csv('data/city_bikes.csv')
df_bike.head()

,Unnamed: 0,latitude,longitude,free_bikes,stationsid
0,0,43.665269,-79.319796,15,fb337bbed72e2be090071e199899b2be
1,1,43.671420,-79.445947,4,4ff88d5880e71aa40d34cfe5d09b0ca7
2,2,43.666224,-79.317693,19,a09c67c0b419654d907c9134b108e328
3,3,43.653236,-79.376716,9,d6a9daee68070a8b106cfb598d81308c
4,4,43.663722,-79.380288,2,8f8af40d9388c8a3962559e8681d3db7


In [6]:
def stations_location(ll_value):
        
    url = f'https://api.foursquare.com/v3/places/search?query=Bar&ll={ll_value}&radius=1000&fields=fsq_id%2Cname%2Clocation%2Ccategories%2Cpopularity%2Crating&sort=POPULARITY'
    api_key = os.environ["FOURSQUARE_API_KEY"]
    headers = {
        "accept": "application/json",
        "Authorization": api_key
               }

    response = requests.get(url, headers=headers).json()
    

    list_of_dict = []
    for poi in response['results']:
        categories = poi.get('categories', None)
        cat_id = categories[0]['id'] if len(categories) > 0 else None
        category_name = categories[0]['name'] if len(categories) > 0 else None

        loci_poi_dict = {
              'lat-long': f'{ll_value}'
            , 'fsq_id': poi['fsq_id']
            , 'cat_id': cat_id
            , 'category_name': category_name
            , 'categories': categories
            , 'name': poi.get('name', None)
            , 'address': poi.get('location', {}).get('formatted_address', None)
            , 'city': poi.get('location', {}).get('locality', None)
            , 'country': poi.get('location', {}).get('country', None)
            , 'popularity': poi.get('popularity', None)
            , 'rating': poi.get('rating', None)

        }
        list_of_dict.append(loci_poi_dict)
    return pd.DataFrame(list_of_dict)

In [4]:
df_bike.dropna(subset=['latitude', 'longitude'], inplace=True)

df_bike['ll'] = df_bike['latitude'].astype(str) + ',' + df_bike['longitude'].astype(str)
df_bike = df_bike[df_bike['ll'] != '0.0,0.0']
print(df_bike.shape)
print(df_bike.dtypes)
df_bike.head()



(656, 6)
Unnamed: 0      int64
latitude      float64
longitude     float64
free_bikes      int64
stationsid     object
ll             object
dtype: object


,Unnamed: 0,latitude,longitude,free_bikes,stationsid,ll
0,0,43.665269,-79.319796,15,fb337bbed72e2be090071e199899b2be,"43.665269,-79.319796"
1,1,43.671420,-79.445947,4,4ff88d5880e71aa40d34cfe5d09b0ca7,"43.67142,-79.445947"
2,2,43.666224,-79.317693,19,a09c67c0b419654d907c9134b108e328,"43.666224,-79.317693"
3,3,43.653236,-79.376716,9,d6a9daee68070a8b106cfb598d81308c,"43.653236,-79.376716"
4,4,43.663722,-79.380288,2,8f8af40d9388c8a3962559e8681d3db7,"43.663722,-79.380288"


In [5]:
ll_bike = df_bike['ll']
ll_bike.head()

0    43.665269,-79.319796
1     43.67142,-79.445947
2    43.666224,-79.317693
3    43.653236,-79.376716
4    43.663722,-79.380288
Name: ll, dtype: object

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [7]:
df_list_fsq = []
for i in ll_bike:
    next = stations_location(i)
    df_list_fsq.append(next)

Put your parsed results into a DataFrame

In [8]:
df_fsq = pd.DataFrame()
x = pd.DataFrame()

for df_i in df_list_fsq:
    x = pd.concat([df_fsq, df_i])
    df_fsq = x

In [9]:
df_fsq.shape

(5797, 11)

In [10]:
df_fsq.head()

,lat-long,fsq_id,cat_id,category_name,categories,name,address,city,country,popularity,rating
0,"43.665269,-79.319796",4b12cd56f964a5206d8e23e3,13049,Diner,"[{'id': 13049, 'name': 'Diner', 'icon': {'pref...",Occasions Restaurant,"30 Eastwood Rd (Gainsborough Road), Toronto ON...",Toronto,CA,0.963599,7.4
1,"43.665269,-79.319796",4da22f3ad686b60c2bb1b428,13018,Pub,"[{'id': 13018, 'name': 'Pub', 'icon': {'prefix...",Queen's Head Pub,"1214 Queen St E (Leslie St), Toronto ON M4M 1L7",Toronto,CA,0.962753,5.6
2,"43.665269,-79.319796",54b47574498e6b087d957fa2,13003,Bar,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",Eulalie's Corner Store,"1438 Gerrard St E (Ashdale), Toronto ON M4L 1Z8",Toronto,CA,0.957616,6.9
3,"43.665269,-79.319796",60ce6c2fe180036cc18a8b13,13010,Dive Bar,"[{'id': 13010, 'name': 'Dive Bar', 'icon': {'p...",Betty's East,"1301 Queen St E, Toronto ON M4L 1C2",Toronto,CA,0.954275,NaN
4,"43.665269,-79.319796",4ea9b2c0cc217eb08e070a93,13031,Burger Joint,"[{'id': 13031, 'name': 'Burger Joint', 'icon':...",Lazy Daisy's Cafe,"1515 Gerrard St E (at Coxwell Ave), Toronto ON...",Toronto,CA,0.939122,7.4


In [22]:
# Save csv

filepath = Path('data/data_Foursquare.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  

df_fsq.to_csv(filepath, index=False) 
df_fsq.head(50)

,lat-long,fsq_id,cat_id,category_name,categories,name,address,city,country,popularity,rating
0,"43.665269,-79.319796",4b12cd56f964a5206d8e23e3,13049,Diner,"[{'id': 13049, 'name': 'Diner', 'icon': {'pref...",Occasions Restaurant,"30 Eastwood Rd (Gainsborough Road), Toronto ON...",Toronto,CA,0.963599,7.4
1,"43.665269,-79.319796",4da22f3ad686b60c2bb1b428,13018,Pub,"[{'id': 13018, 'name': 'Pub', 'icon': {'prefix...",Queen's Head Pub,"1214 Queen St E (Leslie St), Toronto ON M4M 1L7",Toronto,CA,0.962753,5.6
2,"43.665269,-79.319796",54b47574498e6b087d957fa2,13003,Bar,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",Eulalie's Corner Store,"1438 Gerrard St E (Ashdale), Toronto ON M4L 1Z8",Toronto,CA,0.957616,6.9
3,"43.665269,-79.319796",60ce6c2fe180036cc18a8b13,13010,Dive Bar,"[{'id': 13010, 'name': 'Dive Bar', 'icon': {'p...",Betty's East,"1301 Queen St E, Toronto ON M4L 1C2",Toronto,CA,0.954275,NaN
4,"43.665269,-79.319796",4ea9b2c0cc217eb08e070a93,13031,Burger Joint,"[{'id': 13031, 'name': 'Burger Joint', 'icon':...",Lazy Daisy's Cafe,"1515 Gerrard St E (at Coxwell Ave), Toronto ON...",Toronto,CA,0.939122,7.4
5,"43.665269,-79.319796",4fd52f42e4b0b916eb02ab1b,13064,Pizzeria,"[{'id': 13064, 'name': 'Pizzeria', 'icon': {'p...",O Sushi,"6 Coxwell Ave (Queen), Toronto ON M4L 3A7",Toronto,CA,0.935294,7.9
6,"43.665269,-79.319796",4b6bb0cef964a520b6152ce3,10032,Night Club,"[{'id': 10032, 'name': 'Night Club', 'icon': {...",Duke of York,"1225 Queen St E (at Leslie St), Toronto ON M4M...",Toronto,CA,0.930903,5.9
7,"43.665269,-79.319796",584b6fef65e7c76fc4e5c627,13003,Bar,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",Lloyd's on Queen,"1298 Queen St E, Toronto ON M4L 1C4",Toronto,CA,0.894416,NaN
8,"43.665269,-79.319796",4b0ff2a5f964a520a16623e3,13018,Pub,"[{'id': 13018, 'name': 'Pub', 'icon': {'prefix...",Murphy's Law Irish Pub Inc,"1702 Queen St E (at Kingston Rd.), Toronto ON ...",Toronto,CA,0.882665,7.1
9,"43.665269,-79.319796",5f5aa25c6fa9260f1e84871c,13276,Sushi Restaurant,"[{'id': 13276, 'name': 'Sushi Restaurant', 'ic...",Toto Sushi,"1455 Gerrard St E (Craven Rd), Toronto ON M4L 1Z9",Toronto,CA,0.864011,NaN


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [11]:
def stations_location_yelp(lat,long):
  
    url = f'https://api.yelp.com/v3/businesses/search?latitude={lat}&longitude={long}&radius=1000&categories=bars&attributes=name%2C&sort_by=rating&limit=20'
    api_key_yelp = os.environ["YELP_API_KEY"]
    headers = {
        "accept": "application/json",
        "Authorization": f'Bearer {api_key_yelp}'
               }

    response = requests.get(url, headers=headers).json()
    

    list_of_dict = []
    for poi in response['businesses']:
        loci_poi_dict = {
              'lat-long': f'{lat}, {long}'
            , 'category_name': poi['categories'][0]['alias']
            , 'categories': poi['categories']
            , 'name': poi['name']
            , 'latitude': poi['coordinates']['latitude']
            , 'longitude': poi['coordinates']['longitude']
            , 'address': poi['location']['display_address']
            , 'city': poi['location']['city']
            , 'country': poi['location']['country']
            , 'rating': poi.get('rating')
        }
        list_of_dict.append(loci_poi_dict)
    return pd.DataFrame(list_of_dict)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [13]:
df_list_yelp = []
for i in ll_bike:
    coords=re.split('[,]',i)
    next = stations_location_yelp(coords[0],coords[1])
    #print(coords[0] + ' perica ' + coords[1])
    # print(i.get[0],i.get[1])
    df_list_yelp.append(next)   

Put your parsed results into a DataFrame

In [14]:
# Refresh df objects to avoid duplication
df_yelp = pd.DataFrame()
x = pd.DataFrame()

# Generate full df object with all locations through concatenation 
for df_i in df_list_yelp:
    x = pd.concat([df_yelp, df_i])
    df_yelp = x

In [15]:
df_yelp.shape

(10086, 10)

In [16]:
df_yelp.head()

,lat-long,category_name,categories,name,latitude,longitude,address,city,country,rating
0,"43.665269, -79.319796",bakeries,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",Le Conciliabule,43.671365,-79.327225,"[1300 Gerrard Street E, Toronto, ON M4L 1Y7, C...",Toronto,CA,4.5
1,"43.665269, -79.319796",pubs,"[{'alias': 'pubs', 'title': 'Pubs'}]",Eulalie's Corner Store,43.672539,-79.321732,"[1438 Gerrard Street E, Toronto, ON M4L 1Z8, C...",Toronto,CA,4.5
2,"43.665269, -79.319796",cafes,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",Black Pony,43.672410,-79.320400,"[1481 Gerrard Street E, Toronto, ON M4L 2A3, C...",Toronto,CA,4.5
3,"43.665269, -79.319796",bars,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",Lake Inez,43.672340,-79.320640,"[1471 Gerrard Street E, Toronto, ON M4L 2A1, C...",Toronto,CA,4.5
4,"43.665269, -79.319796",cocktailbars,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",Lloyd's On Queen,43.663801,-79.328356,"[1298 Queen St E, Toronto, ON M4L 1C4, Canada]",Toronto,CA,5.0


In [17]:
# Save csv

filepath = Path('data/data_Yelp.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  

df_yelp.to_csv(filepath, index=False) 
df_yelp.head(50)

,lat-long,category_name,categories,name,latitude,longitude,address,city,country,rating
0,"43.665269, -79.319796",bakeries,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",Le Conciliabule,43.671365,-79.327225,"[1300 Gerrard Street E, Toronto, ON M4L 1Y7, C...",Toronto,CA,4.5
1,"43.665269, -79.319796",pubs,"[{'alias': 'pubs', 'title': 'Pubs'}]",Eulalie's Corner Store,43.672539,-79.321732,"[1438 Gerrard Street E, Toronto, ON M4L 1Z8, C...",Toronto,CA,4.5
2,"43.665269, -79.319796",cafes,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",Black Pony,43.672410,-79.320400,"[1481 Gerrard Street E, Toronto, ON M4L 2A3, C...",Toronto,CA,4.5
3,"43.665269, -79.319796",bars,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",Lake Inez,43.672340,-79.320640,"[1471 Gerrard Street E, Toronto, ON M4L 2A1, C...",Toronto,CA,4.5
4,"43.665269, -79.319796",cocktailbars,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",Lloyd's On Queen,43.663801,-79.328356,"[1298 Queen St E, Toronto, ON M4L 1C4, Canada]",Toronto,CA,5.0
5,"43.665269, -79.319796",bowling,"[{'alias': 'bowling', 'title': 'Bowling'}, {'a...",Shamrock Bowl and Restobar,43.673290,-79.319790,"[280 Coxwell Avenue, Toronto, ON M4L 3B6, Canada]",Toronto,CA,4.0
6,"43.665269, -79.319796",sportsbars,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",The Duke Live,43.663149,-79.330307,"[1225 Queen Street E, Toronto, ON M4M 1L6, Can...",Toronto,CA,3.5
7,"43.665269, -79.319796",pubs,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",JP Restaurant,43.673030,-79.319550,"[270 Coxwell Avenue, Toronto, ON M4L 3B6, Canada]",Toronto,CA,3.5
8,"43.665269, -79.319796",bars,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",The Thirsty Loon,43.672394,-79.319906,"[1489 Gerrard Street E, Toronto, ON M4L 2A4, C...",Toronto,CA,4.0
9,"43.665269, -79.319796",mediterranean,"[{'alias': 'mediterranean', 'title': 'Mediterr...",Toronto Beach Club,43.663207,-79.310013,"[1681 Lake Shore Boulevard E, Toronto, ON M4L ...",Toronto,CA,3.0


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

In [27]:
per_station_poi_fsq=df_fsq.groupby('lat-long').fsq_id.count()
per_station_poi_fsq.describe()

count    638.000000
mean       9.086207
std        2.242818
min        1.000000
25%       10.000000
50%       10.000000
75%       10.000000
max       10.000000
Name: fsq_id, dtype: float64

In [30]:
per_station_poi_yelp=df_yelp.groupby('lat-long').category_name.count()
per_station_poi_yelp.describe()

count    638.000000
mean      15.808777
std        6.472581
min        1.000000
25%       11.250000
50%       20.000000
75%       20.000000
max       20.000000
Name: category_name, dtype: float64

When I compare FSQ and Yelp, I find FSQ much more intuitive to use.
There is a much larger selection of components that I can include in FSQ. The only advantage I've seen of Yelp is that within its GET method, you can set a limit on how many POIs you want to display per location. I limited Yeal to 20 (max 20), which is certainly more than FSQ (max 10).

Get the top 10 restaurants according to their rating

In [22]:
#df_fsq.head()
df_fsq.sort_values(by=['rating'], ascending=False).head(10)

,lat-long,fsq_id,cat_id,category_name,categories,name,address,city,country,popularity,rating
3,"43.681944,-79.390556",52ced2c9498e67e1059cf3b2,13059,Juice Bar,"[{'id': 13059, 'name': 'Juice Bar', 'icon': {'...",Greenhouse Juice Co,"5 MacPherson Ave (Yonge St.), Toronto ON M5R 1W7",Toronto,CA,0.92944,9.2
3,"43.6846177,-79.39245919999999",52ced2c9498e67e1059cf3b2,13059,Juice Bar,"[{'id': 13059, 'name': 'Juice Bar', 'icon': {'...",Greenhouse Juice Co,"5 MacPherson Ave (Yonge St.), Toronto ON M5R 1W7",Toronto,CA,0.92944,9.2
7,"43.669903,-79.387802",5bd2379cbcbf7a0039a2d7b9,13003,Bar,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",Storm Crow Manor,"580 Church St, Toronto ON M4Y 2E5",Toronto,CA,0.968008,9.1
3,"43.671526,-79.44874",5753753b498eeb535c53aed5,13003,Bar,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",The Greater Good Bar,"229 Geary Ave (at Dufferin St), Toronto ON M6H...",Toronto,CA,0.946038,9.1
5,"43.671944,-79.387778",5bd2379cbcbf7a0039a2d7b9,13003,Bar,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",Storm Crow Manor,"580 Church St, Toronto ON M4Y 2E5",Toronto,CA,0.968008,9.1
9,"43.663102,-79.373181",5bd2379cbcbf7a0039a2d7b9,13003,Bar,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",Storm Crow Manor,"580 Church St, Toronto ON M4Y 2E5",Toronto,CA,0.968008,9.1
0,"43.672453,-79.338259",52ec057a11d23d4bcd3e74e7,13029,Brewery,"[{'id': 13029, 'name': 'Brewery', 'icon': {'pr...",Left Field Brewery,36 Wagstaff Dr (Greenwood Ave. and Gerrard Str...,Toronto,CA,0.975393,9.1
9,"43.66663,-79.38148",5bd2379cbcbf7a0039a2d7b9,13003,Bar,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",Storm Crow Manor,"580 Church St, Toronto ON M4Y 2E5",Toronto,CA,0.968008,9.1
3,"43.6741219,-79.4352208",5753753b498eeb535c53aed5,13003,Bar,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",The Greater Good Bar,"229 Geary Ave (at Dufferin St), Toronto ON M6H...",Toronto,CA,0.946038,9.1
4,"43.665327,-79.43235",5753753b498eeb535c53aed5,13003,Bar,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",The Greater Good Bar,"229 Geary Ave (at Dufferin St), Toronto ON M6H...",Toronto,CA,0.946038,9.1


In [23]:
#df_yelp.head()
df_yelp.sort_values(by=['rating'], ascending=False).head(10)

,lat-long,category_name,categories,name,latitude,longitude,address,city,country,rating
18,"43.636804, -79.408832",bars,"[{'alias': 'bars', 'title': 'Bars'}]",Sensorium,43.644686,-79.398242,"[525 King Street W, Toronto, ON M5V, Canada]",Toronto,CA,5.0
11,"43.66587, -79.33443",cuban,"[{'alias': 'cuban', 'title': 'Cuban'}, {'alias...",Bar Habana,43.668997,-79.337163,"[1030 Gerrard Street E, Toronto, ON M4M 1Z5, C...",Toronto,CA,5.0
5,"43.6702236, -79.4360232",cocktailbars,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",915 Dupont,43.670099,-79.429762,"[915 Dupont Street, Toronto, ON M6H 1Z1, Canada]",Toronto,CA,5.0
9,"43.649722, -79.416944",musicvenues,"[{'alias': 'musicvenues', 'title': 'Music Venu...",Poetry Jazz Café,43.643768,-79.421750,"[1078 Queen Street W, Toronto, ON M6J 1H8, Can...",Toronto,CA,5.0
1,"43.66587, -79.33443",tapasmallplates,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",East End Vine,43.666996,-79.344665,"[817 Gerrard Street E, Toronto, ON M4M 1Y8, Ca...",Toronto,CA,5.0
8,"43.651238, -79.43868",bars,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",BSMT254,43.651203,-79.440232,"[254 Lansdowne Avenue, Toronto, ON M6H 3X9, Ca...",Toronto,CA,5.0
6,"43.651238, -79.43868",bars,"[{'alias': 'bars', 'title': 'Bars'}]",Blue Bird Bar,43.652270,-79.447950,"[2072 Dundas Street W, Toronto, ON M6R 1W9, Ca...",Toronto,CA,5.0
6,"43.66587, -79.33443",cocktailbars,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",Wei Bar,43.661560,-79.339050,"[1014 Queen St E, Toronto, ON M4M 1K1, Canada]",Toronto,CA,5.0
1,"43.649722, -79.416944",cocktailbars,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",Prequel & Co. Apothecary,43.644141,-79.420339,"[1036 Queen Street W, Toronto, ON M6J 1H7, Can...",Toronto,CA,5.0
10,"43.66587, -79.33443",cocktailbars,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",Lloyd's On Queen,43.663801,-79.328356,"[1298 Queen St E, Toronto, ON M4L 1C4, Canada]",Toronto,CA,5.0
